In [ ]:
!pip install pytorch-nlp
from torchnlp.datasets import imdb_dataset
from tqdm import tqdm
import re
from nltk import tokenize
import nltk
import pickle
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
################################################################################
drive.mount('/content/drive', force_remount=True)
nltk.download('punkt')
train = imdb_dataset(train=True)
test = imdb_dataset(test=True)
################################################################################
class MyDataset(Dataset):
  def __init__(self, x, y):
    self.data = x
    self.labels = y

  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, index):
    return self.data[index], self.labels[index]
################################################################################
class Accuracy:
    """A class to keep track of the accuracy while training"""
    def __init__(self):
        self.correct = 0
        self.total = 0
        
    def reset(self):
        """Resets the internal state"""
        self.correct = 0
        self.total = 0
        
    def update(self, outputs, labels):
        """
        Updates the internal state to later compute the overall accuracy
        
        output: the output of the network for a batch
        labels: the target labels
        """
        _, predicted = torch.max(outputs.data, 1) # predicted now contains the predicted class index/label
        
        self.total += labels.size(0)
        self.correct += (predicted == labels).sum().item() # .item() gets the number, not the tensor
        #self.correct += ((outputs.data > 0.5) == labels).sum().item()

    def compute(self):
        return self.correct/self.total
################################################################################
def eval_accu():
  net.eval()
  eval_accuracy = Accuracy()
  eval_accuracy.reset()        
  with torch.no_grad():
      for eval_data in val_loader:
          # get the data points
          eval_inputs, eval_labels = eval_data
          eval_inputs, eval_labels = eval_inputs.to(device).long(), eval_labels.to(device)
          # forward the data through the network
          eval_outputs = net(eval_inputs)
          
          eval_accuracy.update(eval_outputs, eval_labels)
          
  print("\nTesting Accuracy: {:.2f}%".format(100 * eval_accuracy.compute()))
################################################################################

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
model = pickle.load(open('/content/drive/MyDrive/NLP/w2v_model_gensim.p','rb'))

In [ ]:
model = pickle.load(open('/content/drive/MyDrive/NLP/glove-wiki-gigaword-300-2.p','rb'))

In [ ]:
weights = torch.FloatTensor(model.wv.vectors)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [ ]:
embedding = nn.Embedding.from_pretrained(weights)
# embedding.requires_grad = False

In [ ]:
torch.stack([embedding(torch.tensor(model.vocab['computer'].index)),embedding(torch.tensor(model.vocab['computer'].index))]).size()

In [ ]:
embedding(torch.tensor(model.vocab['computer'].index)) == torch.tensor(model['computer'])

In [ ]:
model.key_to_index[',']
#model.vocab['computer']

1

In [ ]:
torch.tensor(model.key_to_index['computer'])

tensor(951)

In [ ]:
#random.shuffle(train)
vector_size = 300
maxlen = 400
x = torch.zeros((25000,1,maxlen))
y = []
for i in tqdm(range(25000)):
  doc = tokenize.word_tokenize(train[i]['text'])
  fil_doc_w2index = []
  for word in doc:
    try:
      fil_doc_w2index.append(model.key_to_index[word])
    except Exception:
      pass
  if len(fil_doc_w2index)<=maxlen:
    x[i] = torch.cat((torch.LongTensor(fil_doc_w2index),torch.zeros(maxlen-len(fil_doc_w2index)))).unsqueeze(0)
  else:
    x[i] = torch.LongTensor(fil_doc_w2index[-maxlen-1:-1]).unsqueeze(0)
  y.append(torch.tensor(int(train[i]['sentiment']=='pos')))
train_dataset = MyDataset(x,y)


x = torch.zeros((25000,1,maxlen))
y = []
for i in tqdm(range(25000)):
  doc = tokenize.word_tokenize(test[i]['text'])
  fil_doc_w2index = []
  for word in doc:
    try:
      fil_doc_w2index.append(model.key_to_index[word])
    except Exception:
      pass
  if len(fil_doc_w2index)<=maxlen:
    x[i] = torch.cat((torch.LongTensor(fil_doc_w2index),torch.zeros(maxlen-len(fil_doc_w2index)))).unsqueeze(0)
  else:
    x[i] = torch.LongTensor(fil_doc_w2index[-maxlen-1:-1]).unsqueeze(0)
  y.append(torch.tensor(int(test[i]['sentiment']=='pos')))
val_dataset = MyDataset(x,y)

100%|██████████| 25000/25000 [00:39<00:00, 629.36it/s]


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=50, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=50, shuffle=True, num_workers=2)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        weights = torch.FloatTensor(model.wv.vectors)
        self.embedding = nn.Embedding.from_pretrained(weights)
        self.conv1 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (2,vector_size))
        self.conv2 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (3,vector_size))
        self.conv3 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (4,vector_size))
        self.conv4 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (5,vector_size))
        self.fc1 = nn.Linear(400, 2)
        self.dropout = nn.Dropout(0.30)
        self.dropout1 = nn.Dropout(0.30)
        self.dropout2 = nn.Dropout(0.30)
        self.dropout3 = nn.Dropout(0.30)
        self.dropout4 = nn.Dropout(0.30)
        self.dropout5 = nn.Dropout(0.30)
    def forward(self, x):
        x = self.dropout(self.embedding(x))
        x1 = F.relu(self.conv1(x))
        x1 = self.dropout1(F.max_pool2d(x1, (x1.size()[-2],1)))
        x2 = F.relu(self.conv2(x))
        x2 = self.dropout2(F.max_pool2d(x2, (x2.size()[-2],1)))
        x3 = F.relu(self.conv3(x))
        x3 = self.dropout3(F.max_pool2d(x3, (x3.size()[-2],1)))
        x4 = F.relu(self.conv4(x))
        x4 = self.dropout4(F.max_pool2d(x4, (x4.size()[-2],1)))
        x = torch.flatten(torch.cat((x1,x2,x3,x4),-3),start_dim=-3)
        x = self.fc1(self.dropout5(x))
        return x

net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
ind = torch.stack([torch.reshape(torch.stack([torch.tensor(model.key_to_index['computer'])]*200),(1,200))]*100)
embedding(ind).size()

torch.Size([100, 1, 200, 300])

In [ ]:
ind.size()

torch.Size([100, 1, 200])

In [ ]:
#optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.5)
optimizer = optim.Adam(net.parameters(), lr=0.0001)
net = net.float()
net.train()
live_accuracy = Accuracy()

for epoch in range(100):  # loop over the dataset multiple times
    print("\nStarting epoch {}".format(epoch+1))

    live_accuracy.reset()
    total = 0
    running_loss = 0.0

    # to make a beautiful progress bar
    loader = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in loader:
        # get the data points
        inputs, labels = data
        inputs, labels = inputs.to(device).long(), labels.to(device)
        # zero the parameter gradients (else, they are accumulated)
        optimizer.zero_grad()

        # forward the data through the network
        outputs = net(inputs)
        # calculate the loss given the output of the network and the target labels
        loss = criterion(outputs, labels)
        # calculate the gradients of the network w.r.t. its parameters
        loss.backward()
        # Let the optimiser take an optimization step using the calculated gradients
        optimizer.step()
        
        running_loss += loss
        total += outputs.size(0)

        live_accuracy.update(outputs, labels)
        loader.set_description("loss: {:.5f}|acc: {:.2f}%".format(running_loss/total,100 * live_accuracy.compute()))
    eval_accu()
    net.train()

print('Finished Training')


Starting epoch 1


loss: 0.01328|acc: 59.37%: 100%|██████████| 500/500 [00:15<00:00, 31.29it/s]



Testing Accuracy: 78.00%

Starting epoch 2


loss: 0.00989|acc: 77.28%: 100%|██████████| 500/500 [00:15<00:00, 31.77it/s]



Testing Accuracy: 83.00%

Starting epoch 3


...


Testing Accuracy: 88.06%

Starting epoch 14


loss: 0.00532|acc: 89.08%: 100%|██████████| 500/500 [00:15<00:00, 32.27it/s]



Testing Accuracy: 88.31%

Starting epoch 15


loss: 0.00515|acc: 89.15%:  25%|██▍       | 123/500 [00:03<00:11, 32.18it/s]


KeyboardInterrupt: ignored

In [ ]:
pickle.dump(net,open('/content/drive/MyDrive/NLP/imdb_sentiment.p','wb'))

In [ ]:
net = pickle.load(open('/content/drive/MyDrive/NLP/imdb_sentiment.p','rb'))

In [ ]:
net.eval()
accuracy = Accuracy()
accuracy.reset()
# Gradients are calculated on the forward pass for every iteration.
# As we do not need gradients now, we can disable the calculation.
with torch.no_grad():
    for data in tqdm(train_loader):
        # get the data points
        inputs, labels = data
        inputs, labels = inputs.to(device).long(), labels.to(device)
        # forward the data through the network
        outputs = net(inputs)
        
        accuracy.update(outputs, labels)

print("Training Accuracy: {:.2f}%".format(100 * accuracy.compute()))

100%|██████████| 500/500 [00:08<00:00, 57.54it/s]

Training Accuracy: 92.60%


In [ ]:
net.eval()
accuracy = Accuracy()
accuracy.reset()        
with torch.no_grad():
    for data in tqdm(val_loader):
        # get the data points
        inputs, labels = data
        inputs, labels = inputs.to(device).long(), labels.to(device)
        # forward the data through the network
        outputs = net(inputs)
        
        accuracy.update(outputs, labels)
        
print("\nTesting Accuracy: {:.2f}%".format(100 * accuracy.compute()))

100%|██████████| 500/500 [00:08<00:00, 58.67it/s]


Testing Accuracy: 88.30%
